In [1]:
!python3 -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 528.8 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [2]:
# !pip install openai==1.2.3
!pip install openai==1.3.4
!pip install python-dotenv tiktoken
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 798.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 849.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 826.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 863.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 902.3 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 514.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 353.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 880.1 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 579.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 M

In [3]:
from time import time

class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [4]:
import re
# 取得したいURL
url = "https://arxiv.org/abs/2403.08399v1"

identifier = re.search(r'/([^/]+)$', url).group(1)
identifier

'2403.08399v1'

In [5]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [6]:
import requests

def download_pdf(link, save_path):
    response = requests.get(link)
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [7]:
import os

files_to_remove = ['sample.pdf']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} を削除しました。")
    else:
        print(f"{file} は存在しません。")

sample.pdf を削除しました。


In [8]:
url = "https://arxiv.org/pdf/" + identifier

with Timer(prefix="download_pdf"):
    # download_pdf(url, nougat_path)
    download_pdf(url, "sample.pdf")

download_pdf 3.832[s]


In [9]:
pdf_path = "./sample.pdf"

In [10]:
import openai
import pdfplumber
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
import os
import json

load_dotenv()

True

In [11]:
# MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
MODEL_NAME = "gpt-4-0125-preview"
TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

In [12]:
def extract_summary_method_name(model_name, text):
    # 概要と提案手法名を生成する
    prompt = [{'role': 'system', 'content': "Please compile the following text into a research report so that it can be fully understood without any omissions. If a method is proposed, extract its name."}]
    prompt.append({"role" : "system", "content" : 'Please generate the research report to include the following contents.\n"CHALLENGE" The current situation faced by the researcher; it will normally include a Problem Statement, the Motivation, a Hypothesis and/or a Goal.\n"APPROACH" How they intend to carry out the investigation, comments on a theoretical model or framework.\n"OUTCOME" Overall conclusion that should reject or support the research hypothesis.'})
    
    prompt.append({"role" : "system", "content" : 'Please create a research report by referencing the example research report below, structuring it around three distinct aspects in order: "Challenge," "Approach," and "Outcome."\n"Challenge" Existing statistical phrasal orhierarchical machine translation system relies on a large set of translation rules which results in engineering challenges. \n"Approach": The proposed method consistently outperforms existing methods in BLEU on various low-resource language translation tasks with less training data.\n"Outcome" They propose to use factorized grammar from the field of linguistics as more general translation rules from XTAG English Grammar.'})
    
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role": "system", "content": "Think the option as hypothesis. Whether it entails with those premises?"})
    prompt.append({"role" : "system", "content" : "Please explain the research report in a way that is easy to understand for high school students, without making it complicated. It's okay if the explanation becomes lengthy."})
    
    prompt.append({"role": "system", "content": "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    
    prompt.append({"role": "user", "content": 'Based on the input text, generate a JSON containing two different pieces of information. First, use "extract_summary" as the schema to create a section that keys in the result of summarizing the content of the text accurately and completely. Next, use "method_name" as the schema to create a section that keys in the name of the method proposed or used within the text. Combine these pieces of information to output as a single JSON object. This JSON will be formatted as {"extract_summary": [the result of summarizing the text content], "method_name": [the name of the method proposed or used within the text]}.'})
    
    prompt.append({"role": "user", "content": f"Input text: {text}"})
    prompt.append({"role": "user", "content": f"Include:\n- Overview\n- Novelty\n- Methodology\n- Results"})
    """
    システム
    あなたは以下の text を過不足なく理解できるように調査報告書としてまとめ、提案されている手法がある場合はその名称を抽出してください。
    調査報告書は以下の内容を含む形で生成してください\n
    • CHALLENGE: The current situation faced by the researcher; it will normally include a Problem Statement, the Motivation, a Hypothesis and/or a Goal.\n
    • APPROACH: How they intend to carry out the investigation, comments on a theoretical model or framework.\n
    • OUTCOME: Overall conclusion that should reject or support the research hypothesis.

    調査報告書は以下の例を参考に「課題」「アプローチ」「結果」の3つの異なる側面を順番に作成してください。概要: [ACLSum: A New Dataset for Aspect-based Summarization of Scientific Publications](https://arxiv.org/abs/2403.05303v1)のデータセット例を一つ
    出力は Markdown 形式にしてください。
    
    結果は日本語でなければならない。
    user
    入力されたテキストに基づき、二つの異なる情報を含むJSONを生成します。最初に、"extract_summary"をスキーマとして使用し、テキストの内容を過不足なく概要としてまとめた結果をキーとする部分を生成します。次に、"method_name"をスキーマとして使用し、テキスト内で提案されたり使用されている手法名をキーとする部分を生成します。これらの情報を組み合わせて、一つのJSONオブジェクトとして出力します。このJSONは、{"extract_summary": [テキストの内容を概要としてまとめた結果], "method_name": [テキスト内で提案されたり使用されている手法名]}の形式で表されます。
    
    入力されたテキスト: {text}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    summary_method_name_str = response.choices[0].message.content
    # print(summary_method_name_str)
    
    # JSON形式の文字列を辞書に変換
    summary_method_name = json.loads(summary_method_name_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "summary": summary_method_name["extract_summary"],
        "method_name": summary_method_name["method_name"],
    }


In [13]:
file_path = 'sample.pdf'  # PDFファイルのパス

In [14]:
# PDFファイルを読み込み、テキストに変換する関数
def pdf_to_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        return ''.join(page.extract_text() for page in pdf.pages)


In [15]:
with Timer(prefix="pdf_to_text"):
    paper_text = pdf_to_text(file_path)

pdf_to_text 1.305[s]


In [16]:
print(paper_text)

SYSTEM FOR SYSTEMATIC LITERATURE REVIEW USING
MULTIPLE AI AGENTS: CONCEPT AND AN EMPIRICAL
EVALUATION
APREPRINT
AbdulMalikSami ZeeshanRasheed
TampereUniversity TampereUniversity
malik.sami@tuni.fi zeeshan.rasheed@tuni.fi
Kai-KristianKemell MuhammadWaseem TerhiKilamo
UniversityofHelsinki JyväskyläUniversity TampereUniversity
kai-kristian.kemell@helsinki.fi muhammad.m.waseem@jyu.fi terhi.kilamo@tuni.fi
MikaSaari KariSystä AnhNguyenDuc
TampereUniversity TampereUniversity UniversityofSouthEasternNorway
mika.saari@tuni.fi kari.systa@tuni.fi Anh.Nguyen.duc@usn.no
PekkaAbrahamsson
TampereUniversity
pekka.abrahamsson@tuni.fi
March14,2024
ABSTRACT
SystematicLiteratureReviews(SLRs)havebecomethefoundationofevidence-basedstudies,en-
abling researchers to identify, classify, and combine existing studies based on specific research
questions. ConductinganSLRislargelyamanualprocess. Overthepreviousyears,researchers
havemadesignificantprogressinautomatingcertainphasesoftheSLRprocess,aimingtoreduce
thee

In [17]:
# 概要と提案手法名抽出の実行
with Timer(prefix="summary_method_name"):
    summary_method_name = extract_summary_method_name(MODEL_NAME, paper_text)

summary_method_name 25.661[s]


In [18]:
print(summary_method_name["summary"])

本研究では、従来の手作業による時間がかかる文献レビューのプロセスを自動化するために、複数のAIエージェントを活用した新しいモデルを提案しています。このモデルは、大規模言語モデル（LLMs）の能力を用いて、研究者が入力したトピックから関連する学術論文を探索し、選択した論文の要約と分析を行うことで、文献レビューを全自動で行うことができます。このプロセスにより、文献レビューに必要な時間と労力を大幅に削減し、精度と効率を向上させています。実際の評価では、ソフトウェアエンジニアリングの研究者たちがこのモデルをテストし、高い評価とともに、さらなる改善のためのフィードバックを提供しました。今後は、より多くの実務家と研究者による評価を行い、2024年のSANERカンファレンスでの発表を通じて、モデルの改善と実用化を目指しています。


In [19]:
print(summary_method_name["method_name"])

LLMを利用した複数AIエージェントモデル


In [20]:
def explain_method_algorithm(model_name, text, method_name):
    # アルゴリズムの説明
    prompt = [{'role': 'system', 'content': "Please explain the algorithm of the method name from the following text in detail, using both sentences and formulas. Carefully describe the mechanism in order, ensuring that it can be understood and implemented. Design the process flow in a way that allows for the algorithm to be implemented without any omissions or excess."}]
    prompt.append({"role" : "system", "content" : "Describe the algorithm in detail, explaining what it aims to achieve, how it processes to accomplish this, and how exactly these processes are carried out, regardless of the length of the explanation. Just ensure it is accurate."})
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role" : "system", "content" : "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    prompt.append({"role": "system", "content": 'Please generate a JSON from the following input text. Use "method" as the schema, and for the key, use "the detailed explanation of the processing of the method_name algorithm in simple language". Generate it in the format of {"method": "the result of a detailed explanation of the method_names algorithm described in simple language"}.'})
    
    prompt.append({"role": "user", "content": 'Generate a JSON from the following input text. Use "method" as the schema, and use the judgment result as the key, to create it in the format {"method": the result of grouping the search_query based on relevance into a list format that can be used in Python}.'})
    
    prompt.append({"role": "user", "content": f"Input text: {text}"})
    prompt.append({"role": "user", "content": f"method name: {method_name}"})
    
    """
    システム
    あなたは以下の text から method name のアルゴリズムを順番に過不足なく文章と数式で丁寧に順番に仕組みが理解でき、実装をするための処理の流れを設計できるように説明してください。
    アルゴリズムの説明は、何を実現するために、どのように処理を実行し、その処理はどのように実行されるのかをどれだけ長くなってもよいのでとにかく正確に説明してください。
    出力は Markdown 形式にしてください。
    
    結果は日本語でなければならない。
    以下の入力テキストからJSONを生成してください。スキーマには "method"、キーには"text から method_name のアルゴリズムを平易な文章で処理内容を詳細に説明した内容"を使ってください。"method": "method_name のアルゴリズムを平易な文章で処理内容を詳細に説明した内容した結果}'}の形式で生成してください。
    user
    
    入力されたテキスト: {text}
    method name: {method_name}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    method = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    method_str = method.choices[0].message.content
    
    # JSON形式の文字列を辞書に変換
    method_algorithm = json.loads(method_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "method": method_algorithm["method"],
    }


In [21]:
with Timer(prefix="explain_method_algorithm"):
    method_algorithm = explain_method_algorithm(MODEL_NAME, paper_text, str(summary_method_name["method_name"]))

explain_method_algorithm 16.346[s]


In [22]:
print(method_algorithm["method"])

LLMを利用した複数AIエージェントモデルは、システマティックレビュー(SLR)のプロセスを自動化するために開発された新しい手法です。このモデルは、大規模言語モデル(LLM)の能力を活用して、研究者が入力したトピックに基づいて適切な検索文字列を生成し、関連する学術論文を効率的に検索し、選択します。その後、包括的および排除的なフィルタリングプロセスを適用し、特定の研究領域に関連するタイトルに焦点を当てます。モデルはこれらの論文の要約を自動的に生成し、研究分野に直接関連するもののみを保持します。最終段階では、モデルは定義された研究質問に関連して選択された論文の徹底的な分析を行います。この手法は、SLRを行うために必要な時間と労力を大幅に削減し、包括性と精度を確保することを目的としています。


In [23]:
def generate_pseudocode_for_method(model_name, algorithm):
    # アルゴリズムの説明
    prompt = [{'role': 'system', 'content': "Based on the description of the following algorithm, please create a comprehensive pseudo-implementation code in Python without omitting any details."}]
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role": "system", "content": "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Comment must be in Japanese."})
    prompt.append({"role": "system", "content": 'Please generate a JSON from the following input text. Use "code" as the schema, and for the key, use "the result of generating code that executes the algorithm of algorithm in Python". Generate it in the format of {"code": "the result of reproducing the algorithm algorithm in Python code"}.'})
        
    prompt.append({"role": "user", "content": f"algorithm: {algorithm}"})
    
    """
    システム
    あなたは以下の algorithm の説明を基にpythonの疑似実装コードを過不足なく作成してください。
    出力は Markdown 形式にしてください。
    
    コメントは日本語でなければならない。
    以下の入力テキストからJSONを生成してください。スキーマには "code"、キーには"algorithm のアルゴリズムをpythonのコードで動くようにコードを生成した結果"を使ってください。"code": "algorithm のアルゴリズムをpythonのコードで再現した結果"}'}の形式で生成してください。
    user
    
    algorithm: {algorithm}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    code_res = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    code_str = code_res.choices[0].message.content
    print(code_str)
    
    # JSON形式の文字列を辞書に変換
    code = json.loads(code_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "code": code["code"],
    }


In [24]:
with Timer(prefix="generate_pseudocode_for_method"):
    code_method = generate_pseudocode_for_method(MODEL_NAME, str(method_algorithm["method"]))


{
  "code": "# LLMを利用した複数AIエージェントモデルの実装\n\n# 必要なライブラリのインポート\nimport openai\n\n# OpenAIのAPIキー設定\nopenai.api_key = 'your_api_key_here'\n\n# SLRプロセスの自動化\n\nclass SLRAutomation:\n    def __init__(self, topic):\n        self.topic = topic\n\n    # 適切な検索文字列を生成するメソッド\n    def generate_search_string(self):\n        # APIを利用して検索文字列を生成\n        response = openai.Completion.create(\n          engine=\"davinci\",\n          prompt=f\"研究トピック: {self.topic}に基づく適切な検索文字列を生成してください。\",\n          max_tokens=100\n        )\n        search_string = response.choices[0].text.strip()\n        return search_string\n\n    # 関連する学術論文を検索し選択するメソッド\n    def find_relevant_papers(self, search_string):\n        # 仮の実装、実際には検索APIやデータベースを利用して関連論文を検索\n        return ['paper1', 'paper2', 'paper3']\n\n    # 論文の要約を生成するメソッド\n    def generate_summaries(self, papers):\n        summaries = []\n        for paper in papers:\n            # APIを利用して論文の要約を生成\n            response = openai.Completion.create(\n              engine=\"d

In [25]:
print(code_method["code"])

# LLMを利用した複数AIエージェントモデルの実装

# 必要なライブラリのインポート
import openai

# OpenAIのAPIキー設定
openai.api_key = 'your_api_key_here'

# SLRプロセスの自動化

class SLRAutomation:
    def __init__(self, topic):
        self.topic = topic

    # 適切な検索文字列を生成するメソッド
    def generate_search_string(self):
        # APIを利用して検索文字列を生成
        response = openai.Completion.create(
          engine="davinci",
          prompt=f"研究トピック: {self.topic}に基づく適切な検索文字列を生成してください。",
          max_tokens=100
        )
        search_string = response.choices[0].text.strip()
        return search_string

    # 関連する学術論文を検索し選択するメソッド
    def find_relevant_papers(self, search_string):
        # 仮の実装、実際には検索APIやデータベースを利用して関連論文を検索
        return ['paper1', 'paper2', 'paper3']

    # 論文の要約を生成するメソッド
    def generate_summaries(self, papers):
        summaries = []
        for paper in papers:
            # APIを利用して論文の要約を生成
            response = openai.Completion.create(
              engine="davinci",
              prompt=f"以下の論文の要約を生成してください: {pap